## 1. Connect to the workspace

In [ ]:
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")

In [ ]:
#import required libraries
from azure.ai.ml import MLClient

#Enter details of your Azure Machine Learning workspace
subscription_id = ''
resource_group = ''
workspace = ''

#connect to the workspace
ml_client = MLClient(credential, subscription_id, resource_group, workspace)

## 2. Create a compute resource for training

In [ ]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

## 3. Create a Python Script

In [ ]:
%%writefile -a src/sample.py

# import libraries
import mlflow
import argparse
import pandas as pd

def main(args):
    # read data
    df = get_data(args.training_data)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)
    mlflow.log_metric("score", 100)
    print(df.head(2))
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":

    # parse args
    args = parse_args()

    # run main function
    main(args)

## 4. Submit the job

In [ ]:
from azure.ai.ml import command, Input

# define the command
command_job = command(
    code="./src",
    command="python sample.py --training_data ${{inputs.bike_csv}}",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    inputs={
        "bike_csv": Input(
            type="uri_file",
            path="azureml://subscriptions/89524954-8e30-4fa1-b865-1609eeb40581/resourcegroups/ml-rg/workspaces/aml-001/datastores/blobstore/paths/bike-no.csv",
        ),
    },
    compute="cpu-cluster",
)

In [ ]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url